# Pre-Processing

In [1]:
from utils.dataloader import *
from utils.sift import *
from utils.helper import strech_img
from sklearn.model_selection import train_test_split
from utils.hog import *
import pickle

In [2]:
RANDOM_STATE = 66
ROOT_PATH, DATA_PATH, OUT_PATH = get_paths()
print(ROOT_PATH)

/Users/moooooo16/Documents/Computer Vision/Project/elpv


In [3]:
img_path, prob, types, labels= load_data(DATA_PATH)
print(len(img_path), 
      len(prob),
      len(types),
      len(labels))

[0 1 2 3 4 5 6 7] [588 117  56 313 920 178  50 402]
2624 2624 2624 2624


# Class mapping 

|Label | prob | type|
|------|------|-----|
|0     | 0.0  | mono|
|1     | 0.333| mono|
|2     | 0.667| mono|
|3     | 1.0  | mono|
|4     | 0.0  | poly|
|5     | 0.333| poly|
|6     | 0.667| poly|
|7     | 1.0  | poly|

In [4]:
sift = SIFT(DATA_PATH, img_path, processing_funcs=[strech_img])
kps, descriptor, empty_descriptor = sift.calculate_desriptor(mask=None)
print(empty_descriptor)

Calculating descriptors: 100%|██████████| 2624/2624 [00:20<00:00, 126.01it/s]

[]


In [5]:
X_train, X_test, y_train, y_test = train_test_split(descriptor, labels, test_size=0.25, random_state=RANDOM_STATE, stratify= labels)


print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

NameError: name 'descriptor' is not defined

In [6]:
ks = [128, 256, 512, 1024]

kmeans, features = sift.build_features(X_train, ks, 
                               state=RANDOM_STATE,
                               init = 'k-means++',
                               n_init=10,
                               max_iter=300,
                               tol=1e-4,)

Calculating kmeans for k = 128


Building histogram for k = 128: 100%|██████████| 1968/1968 [00:02<00:00, 759.73it/s]


Calculating kmeans for k = 256


Building histogram for k = 256: 100%|██████████| 1968/1968 [00:02<00:00, 680.54it/s]


Calculating kmeans for k = 512


Building histogram for k = 512: 100%|██████████| 1968/1968 [00:03<00:00, 618.19it/s]


Calculating kmeans for k = 1024


Building histogram for k = 1024: 100%|██████████| 1968/1968 [00:04<00:00, 461.96it/s]


In [7]:
test_hist = {}
for k in ks:
    hist = sift.get_features(X_test, kmeans[k], k)
    test_hist[k] = hist

Building histogram for k = 1024: 100%|██████████| 656/656 [00:01<00:00, 474.30it/s]


In [8]:
with open(os.path.join(OUT_PATH, 'sift_X.pkl'), 'wb') as f:
    pickle.dump((features, test_hist), f)
    
with open(os.path.join(OUT_PATH, 'sift_kmeans.pkl'), 'wb') as f:
    pickle.dump(kmeans, f)
    
with open(os.path.join(OUT_PATH, 'sift_y.pkl'), 'wb') as f:
    pickle.dump((y_train, y_test), f)

In [5]:
hog = HOG(DATA_PATH, img_path)
hog_features, hog_imgs = hog.get_hog_features(orient=10,
                                    pix_per_cell= 10,
                                    cell_per_block= 1,
                                    vis = True,
                                    feature_vec=True)

Calculating descriptors: 100%|██████████| 2624/2624 [01:44<00:00, 24.99it/s]


In [6]:
X_train, X_test, y_train, y_test = train_test_split(hog_features, labels, test_size=0.25, random_state=RANDOM_STATE, stratify= labels)


print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

1968 656
1968 656


In [8]:
with open(os.path.join(OUT_PATH, 'hog_X.pkl'), 'wb') as f:
    pickle.dump((X_train, X_test), f)
    
with open(os.path.join(OUT_PATH, 'hog_y.pkl'), 'wb') as f:
    pickle.dump((y_train, y_test), f)